# Baseline

In [20]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")

In [37]:
root = "../models/"
models = ["baseline", "fovnetV1", "baseline2", "multiRes", "fovnetV2", "Cartesian", ]
models = [root + model for model in models]
names = ["Most Salient: Uniform", "Most Salient: Polar-Retinal",
         "Top-3 Salient: Uniform", "Top-3 Salient: Multi-Resolution",
         "Top-3 Salient: Polar-Retinal", "Top-3 Salient: Cartesian-Retinal"]
train = [model + "/training.out" for model in models]
test = [model + "/test.out" for model in models]

In [38]:
def process(train_name, test_name, name):
    df_train = pd.read_csv(train_name, header=None, sep="\t")
    df_test = pd.read_csv(test_name, header=None, sep="\t")

    df_test["Top 1"] = df_test[3].apply(lambda x: x.strip().split()[1]).astype(float)
    df_test["Top 5"] = df_test[4].apply(lambda x: x.strip().split()[1]).astype(float)

    df_train["Top 1"] = df_train[4].apply(lambda x: x.strip().split()[1]).astype(float)
    df_train["Top 5"] = df_train[5].apply(lambda x: x.strip().split()[1]).astype(float)

    def get_accuracy(df, train=True):
        i = 1
        top5_acc = []
        top1_acc = []
        top5_total = 0
        top1_total = 0
        if train:
            flag = "[   0/5005]"
#             flag = "[5000/5005]"
        else:
            flag = "[  0/196]"
#             flag = "[190/196]"
        for index, row in df.iterrows():
            if (flag in row[0]):
                top1_acc.append(top1_total/i)
                top5_acc.append(top5_total/i)
#                 top1_acc.append(row["Top 1"])
#                 top5_acc.append(row["Top 5"])
                top1_total = 0
                top5_total = 0
                i = 0


            top1_total += row["Top 1"]
            top5_total += row["Top 5"]
            i += 1
        return top1_acc[1:], top5_acc[1:]
    
    top1_train, top5_train = get_accuracy(df_train, train = True)
    top1_test, top5_test = get_accuracy(df_test, train = False)
    
    return {"top1":{"train":top1_train, "test":top1_test}, "top5":{"train":top5_train, "test":top5_test}}

In [ ]:
top5_train = []
top5_test = []
top1_train = []
top1_test = []
for i in range(len(models)):
    vals = process(train[i], test[i], names[i])
    top5_train.append(vals["top5"]["train"])
    top5_test.append(vals["top5"]["test"])
    top1_train.append(vals["top1"]["train"])
    top1_test.append(vals["top1"]["test"])

In [ ]:
records = {"top5": {}, "top1": {}}
for i in top5_train:
    plt.plot(i)
    
plt.legend(names)
plt.title("Top 5 Train Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy (%)")
plt.show()

for i,v  in enumerate(top5_test):
    plt.plot(v)
    records["top5"][names[i]] = max(v)
    
plt.legend(names)
plt.title("Top 5 Validation Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy (%)")
plt.show()

for i in top1_train:
    plt.plot(i)
    
plt.legend(names)
plt.title("Top 1 Train Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy (%)")
plt.show()

for i,v in enumerate(top1_test):
    plt.plot(v)
    records["top1"][names[i]] = max(v)

    
plt.legend(names)
plt.title("Top 1 Validation Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy (%)");

In [36]:
records

{'top5': {'Most Salient: Uniform': 63.02849999999999,
  'Top-3 Salient: Uniform': 66.055,
  'Top-3 Salient: Cartesian-Retinal': 65.74249999999999,
  'Most Salient: Polar-Retinal': 57.501,
  'Top-3 Salient: Polar-Retinal': 61.56199999999999,
  'Top-3 Salient: Multi-Resolution': 62.694500000000005},
 'top1': {'Most Salient: Uniform': 37.656,
  'Top-3 Salient: Uniform': 41.425500000000014,
  'Top-3 Salient: Cartesian-Retinal': 41.855000000000004,
  'Most Salient: Polar-Retinal': 33.847,
  'Top-3 Salient: Polar-Retinal': 38.26050000000001,
  'Top-3 Salient: Multi-Resolution': 38.886}}